In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 22:09:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-11-18 22:09:52 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://<your connection string>:5432/<your database-name>"
config = {"user":"<your database user>",
          "password": "<your database password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
video_game_vine_reviews = spark.read.jdbc(url=jdbc_url,table="video_game_vine_reviews", properties=config)
video_game_vine_reviews.show(10)

In [ ]:
software_vine_reviews = spark.read.jdbc(url=jdbc_url, table="software_vine_reviews", properties=config)
software_vine_reviews.show(10)

## **Vine Analysis**

In [ ]:
video_game_vine_reviews.describe().show()

In [ ]:
software_vine_reviews.describe().show()

In [ ]:
from pyspark.sql.functions import round
video_game_reviews_count = video_game_vine_reviews.count()
video_game_helpful_vote = video_game_vine_reviews.groupBy("helpful_votes").count()
video_game_helpful_vote = video_game_helpful_vote.orderBy(video_game_helpful_vote["count"].desc())
video_game_helpful_vote.withColumn('percentage', round(video_game_helpful_vote["count"]/video_game_reviews_count,4)).show()

In [ ]:
software_reviews_count = software_vine_reviews.count()
software_helpful_vote = software_vine_reviews.groupBy("helpful_votes").count()
software_helpful_vote = software_helpful_vote.orderBy(software_helpful_vote["count"].desc())
software_helpful_vote.withColumn('percentage', round(software_helpful_vote["count"]/software_reviews_count,4)).show()

In [ ]:
video_game_star_rating = video_game_vine_reviews.groupBy("star_rating").count()
video_game_star_rating = video_game_star_rating.orderBy(video_game_star_rating["count"].desc())
video_game_star_rating.withColumn('percentage', round(video_game_star_rating["count"]/video_game_reviews_count,4)).show()

In [ ]:
software_star_rating = software_vine_reviews.groupBy("star_rating").count()
software_star_rating = software_star_rating.orderBy(software_star_rating["count"].desc())
software_star_rating.withColumn('percentage', round(software_star_rating["count"]/software_reviews_count,4)).show()